In [ ]:
import wave
import librosa
import os
import subprocess, re
import random

In [ ]:
TST_DATA_DIR = '../../.tstdata'
DATA_DIR = TST_DATA_DIR + '/merged'
OUT_4SEC_DIR  = TST_DATA_DIR + '/4-second'
OUT_AUG_DIR   = TST_DATA_DIR + '/augumented'

In [ ]:
def measure_duration(in_path: str) -> float:
    """Get duration in seconds."""
    cmd = ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "csv=p=0", in_path]
    out = subprocess.run(cmd, capture_output=True, text=True)
    try:
        return float(out.stdout.strip())
    except:
        return 0.0

def measure_rms(in_path: str) -> float:
    """Returns RMS level (dB) of a wav file."""
    cmd = [
        "ffmpeg", "-hide_banner", "-i", in_path,
        "-af", f"highpass=f={9000},volumedetect",
        "-vn", "-f", "null", "-"
    ]
    proc = subprocess.run(cmd, stderr=subprocess.PIPE, text=True)
    match = re.search(r"mean_volume:\s*(-?\d+\.\d+)", proc.stderr)
    if match:
        return float(match.group(1))
    return -25.0  # fa

def ffmpeg_pad_recording(in_path:str, out_path:str, time=4):
    '''
    This monstrosity of a command should have never seen the light of the day. 
    But hey, at least is mostly does what it should ;)
    '''
    # WTF_FFMPEG = f'''
    # ffmpeg -i {in_path} -filter_complex "[0:a]aresample=48000,pan=mono|c0=0.5*c0[a]; \
    #     aevalsrc=random(0):d={time}:s=48000:channel_layout=mono[noise]; \
    #     [noise]volume=0.02[filtered]; \
    #     [a][filtered]amix=inputs=2:duration=longest:dropout_transition=0,atrim=duration={time}[out]"  -map "[out]" -ac 1 -ar 48000 -acodec pcm_s16le {out_path}
    # '''

    rms = measure_rms(in_path) # adjust noise to file noise level
    volume_factor = 10 ** ((rms) / 50) * 0.02
    # volume_factor = rms
    volume_factor = max(0.005, min(volume_factor, 0.05))
    
    duration = measure_duration(in_path)
    FADE_IN = 0.3
    delay_ms = max(0, (duration - FADE_IN) * 1000)

    WTF_FFMPEG_2 = f"""
    ffmpeg -i "{in_path}" -filter_complex "
        [0:a]aresample=48000,pan=mono|c0=0.5*c0, afade=t=out:st={duration-0.06}:d=0.06[a];
        aevalsrc=random(0):d={time}:s=48000:channel_layout=mono[noise];
        [noise]volume={volume_factor},
               afade=t=in:st=0:d={FADE_IN},
               adelay={delay_ms}|{delay_ms}[filtered];
        [a][filtered]amix=inputs=2:duration=longest:dropout_transition=0,
               atrim=duration={time}[out]
    " -map "[out]" -ac 1 -ar 48000 -acodec pcm_s16le -y "{out_path}"
    """
    os.system(WTF_FFMPEG_2)


def pad_recordings(src_root: str, dst_root: str):
    for current_root, dirs, files in os.walk(src_root):
        rel_path = os.path.relpath(current_root, src_root)
        dst_dir = os.path.join(dst_root, rel_path)

        os.makedirs(dst_dir, exist_ok=True)

        # Process wavs
        for f in files:
            src_path = os.path.join(current_root, f)
            dst_path = os.path.join(dst_dir, f)
            ffmpeg_pad_recording(src_path, dst_path)



In [ ]:
pad_recordings(DATA_DIR, OUT_4SEC_DIR)

# Augument

In [ ]:

def rand_prefix_recording(in_path:str, out_path:str, time=4):

    rms = measure_rms(in_path) # adjust noise to file noise level
    volume_factor = 10 ** ((rms) / 50) * 0.02
    # volume_factor = rms
    volume_factor = max(0.005, min(volume_factor, 0.05))
    
    prefix_duration = abs(random.random())
    FADE_OUT = 0.05
    delay_ms = max(0, (prefix_duration - FADE_OUT) * 1000)

    WTF_FFMPEG_2 = f"""
    ffmpeg -i "{in_path}" -filter_complex "
        [0:a]aresample=48000,pan=mono|c0=0.5*c0,adelay={delay_ms}|{delay_ms}[a];
        aevalsrc=random(0):d={prefix_duration}:s=48000:channel_layout=mono[noise_pre];
        [noise_pre]volume={volume_factor},
               afade=t=out:st={prefix_duration-FADE_OUT}:d={FADE_OUT}[filtered_pre];
        [filtered_pre][a]amix=inputs=2:duration=longest:dropout_transition=0,
               atrim=duration={time}[out]
    " -map "[out]" -ac 1 -ar 48000 -acodec pcm_s16le -y "{out_path}"
    """
    os.system(WTF_FFMPEG_2)



In [ ]:
src = TST_DATA_DIR + '/4-second'
dst = TST_DATA_DIR + '/augumented'
for current_root, dirs, files in os.walk(src):
    rel_path = os.path.relpath(current_root, src)
    dst_dir = os.path.join(dst, rel_path)

    os.makedirs(dst_dir, exist_ok=True)

    # Process wavs
    for f in files:
        src_path = os.path.join(current_root, f)
        dst_path = os.path.join(dst_dir, "aug_1_" + f)
        rand_prefix_recording(src_path, dst_path)

In [ ]:
import shutil
from pathlib import Path
from typing import Iterable

def merge_roots(src_roots: Iterable[str], dst_root: str, conflict: str = "rename"):
    """
    Recursively copy files from multiple source roots into a single destination root,
    preserving relative directory structure.

    conflict: "rename" (default) | "overwrite" | "skip"
      - "rename": if a target file exists, create file_base_1.ext, _2, ...
      - "overwrite": overwrite existing files
      - "skip": do not copy files that would collide
    """
    dst_root = Path(dst_root)
    dst_root.mkdir(parents=True, exist_ok=True)

    copied = 0
    skipped = 0
    renamed = 0

    for src_root in src_roots:
        src_root = Path(src_root)
        if not src_root.exists():
            continue
        for src_path in src_root.rglob('*'):
            if src_path.is_dir():
                continue
            # relative path w.r.t this source root
            rel = src_path.relative_to(src_root)
            target_dir = dst_root.joinpath(rel.parent)
            target_dir.mkdir(parents=True, exist_ok=True)
            target_path = target_dir.joinpath(rel.name)

            if not target_path.exists():
                shutil.copy2(src_path, target_path)
                copied += 1
            else:
                if conflict == "overwrite":
                    shutil.copy2(src_path, target_path)
                    copied += 1
                elif conflict == "skip":
                    skipped += 1
                else:  # rename
                    base = target_path.stem
                    ext = target_path.suffix
                    i = 1
                    while True:
                        candidate = target_dir.joinpath(f"{base}_{i}{ext}")
                        if not candidate.exists():
                            shutil.copy2(src_path, candidate)
                            renamed += 1
                            break
                        i += 1

    print(f"merged {len(list(src_roots))} roots -> {dst_root}")
    print(f"copied: {copied}, renamed: {renamed}, skipped: {skipped}")


merge_roots([TST_DATA_DIR+'/4-second', TST_DATA_DIR+'/augumented'], TST_DATA_DIR+'/dataset', conflict="overwrite")

# Dataset split

In [ ]:
import os
import shutil
import random

def split_dataset(root_dir, ratio=0.8):
    root_dir = os.path.abspath(root_dir)
    train_dir = os.path.join(root_dir, "train")
    val_dir = os.path.join(root_dir, "val")

    # Create output directories
    for dir_path in [train_dir, val_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Iterate through label folders
    for label in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label)
        if not os.path.isdir(label_path) or label in ("train", "val"):
            continue

        images = [f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))]
        random.shuffle(images)

        split_point = int(len(images) * ratio)
        train_images = images[:split_point]
        val_images = images[split_point:]

        # Make label folders in train and val dirs
        os.makedirs(os.path.join(train_dir, label), exist_ok=True)
        os.makedirs(os.path.join(val_dir, label), exist_ok=True)

        # Move or copy files
        for img in train_images:
            shutil.copy2(os.path.join(label_path, img), os.path.join(train_dir, label, img))
        for img in val_images:
            shutil.copy2(os.path.join(label_path, img), os.path.join(val_dir, label, img))

        print(f"Label '{label}': {len(train_images)} train, {len(val_images)} val")



In [ ]:
split_dataset(TST_DATA_DIR + '/dataset')